In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import make_classification
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split

# synthetic data with 200 samples/data records and 12 input features/size of the input using make_classification
X, y = make_classification(n_samples=200, n_features=12, random_state=42)

# transformed labels from -1, 1 to 0, 1 for training
# Neural networks common practice to use non-negative labels and also we will be using sigmoid activation function which is [0,1]
labels_transformed = ((y + 1) / 2).astype(int)
# labels_transformed

# Split the data into training (80%) and testing (20% => 0.2) sets
x_train, x_test, y_train, y_test = train_test_split(X,labels_transformed, test_size=0.2)

# Configure the Neural Network autoencoder model
# Input layer with 12 units/nodes (one for each input feature)
input_layer = Input(shape=(12,))

# Encoder layers: Reducing dimensionality from 12 -> 8 -> 4
# output of previous layer is an input to next layer
encoder = Dense(8, activation='relu')(input_layer)
encoded = Dense(4, activation='relu')(encoder)

# Decoder layers: Reconstructing back from 4 -> 8 -> 12
decoder = Dense(8, activation='relu')(encoded)
decoded = Dense(12, activation='sigmoid')(decoder)

# Classification layer: Used the encoded/bottleneck layer to classify
# Output is set to one node and we apply sigmoid activation function which is suitable for binary classification
classifier = Dense(1, activation='sigmoid')(encoded)

# Build the Neural Network autoencoder model
# The model has two outputs: one for reconstruction, one for classification
NN_autoencoder_Model = Model(inputs=input_layer, outputs=[decoded, classifier])

NN_autoencoder_Model.summary()
# Compile the model
# Used two different loss functions: mean_squared_error
#(measures the difference between the original input and the reconstructed output) for reconstruction,
# and binary_crossentropy for classification
# we can also use different functions like mean_absolute_error, mean_squared_logarithmic_error
NN_autoencoder_Model.compile(optimizer='adam',
                    loss=['mean_squared_error', 'binary_crossentropy'],
                    metrics='accuracy')


# Train the model with train set
# The model is trained to do both tasks simultaneously: reconstruct and classify
# epochs =50 we want to do 50 iterations (forward and backward) through the entire data set.
NN_autoencoder_Model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate the model's performance with test set
evaluate_model = NN_autoencoder_Model.evaluate(x_test, y_test)
print(f"Autoencoder Loss: {evaluate_model[1]}, Classification Accuracy: {evaluate_model[2]}")


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 12)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 8)                    104       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 4)                    36        ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 8)                    40        ['dense_1[0][0]']             
                                                                                              